In [284]:
import numpy as np
import math
import cv2  
import numpy as np
import os
import glob

In [285]:
def checkValidState(state):
    if len(state)<16:
        state=state+"".join(["O" for i in range(16-len(state))])
    s=np.array(list(state))
    c1=len(np.where(s=="F")[0])
    c2=len(np.where(s=="S")[0])
    if c1-c2==0 or c1-c2==1:
        return(True)
    else:
        return(False)

In [286]:
def searchSubstring(state,check):
    b=np.array(list(state))
    substring="".join(b[check])
    return(substring)
    

In [287]:
def getWinningWaysOld():
    winningWays=[[0,1,2],[1,2,3],[4,5,6],[5,6,7],[8,9,10],[9,10,11]]
    winningWays=winningWays+[[12,13,14],[13,14,15],[0,4,8],[4,8,12],[1,5,9],[5,9,13]]
    winningWays=winningWays+[[2,6,10],[6,10,14],[3,7,11],[7,11,15]]
    winningWays=winningWays+[[0,5,10],[5,10,15],[3,6,9],[6,9,12],[2,5,8],[7,10,13]]
    winningWays=winningWays+[[1,6,11],[4,9,14]]
    return(winningWays)

def getWinningWays():
    winningWays=[[0,1,2,3],[4,5,6,7],[8,9,10,11],[12,13,14,15]]
    winningWays=winningWays+[[0,4,8,12],[1,5,9,13],[2,6,10,14],[3,7,11,15]]
    winningWays=winningWays+[[0,5,10,15],[3,6,9,12]]
    winningWays=winningWays+[[1,6,11],[4,9,14]]
    winningWays=winningWays+[[2,5,8],[7,10,13]]
    winningWays=winningWays+[[1,4],[2,7],[8,13],[11,14]]
    return(winningWays)

In [288]:
winning_ways=getWinningWays()

In [289]:
def checkStateStatus(state):
    
    if len(state)<16:
        state=state+"".join(["O" for i in range(16-len(state))])
    if not checkValidState(state):
        print("Not a valid state")
        print("State is\t"+state)
        return
    global winning_ways
    for check in winning_ways:
            ans=searchSubstring(state,check)
            
            if len(check)==3:
                if ans=="FFF" or ans=="SSS":
                    return("WIN")
            if len(check)==4:
                if ans=="FFFF" or ans=="SSSS":
                    return("WIN")
            if len(check)==2:
                if ans=="FF" or ans=="SS":
                    return("WIN")
    if "O" not in np.array(list(state)):
        return("DRAW")
    return("ON")

In [290]:
def whoseTurn(s):
    b=np.array(list(s))
    c1=len(np.where(b=="F")[0])
    c2=len(np.where(b=="S")[0])
    if c1==c2:
        return("F")
    if c1-c2==1:
        return("S")
    

In [291]:
def nextMove(state):
    if len(state)<16:
        state=state+"".join(["O" for i in range(16-len(state))])
    b=np.array(list(state))
    player=whoseTurn(state)
    ind=np.where(b=="O")[0]
    arr=[]
    for i in ind:
        b=np.array(list(state))
        if player=="F":
            b[i]="F"
        if player=="S":
            b[i]="S"
        arr.append("".join(b))
    return(arr)
    

In [329]:
def save(state,title=""):
    
    if len(state)<16:
        state=state+"".join(["O" for i in range(16-len(state))])
    if not checkValidState(state):
        print("Not a valid state in save")
        return
    if len(title)==0:
        title=state
    img = np.zeros((800,800,3), np.uint8)
    b=np.array(list(state))
    cv2.rectangle(img, (0,0), (800,800), (255,255,255), -1)
    for i in range(16):
        if b[i]!="O":
            column=i//4
            row=i%4
            
            if b[i]=="F":
                color=(0,255,0)
                cv2.circle(img, (200*row+100,200*column+100), 80,color, -1)
            else:
                color=(0,0,255)
                cv2.circle(img, (200*row+100,200*column+100), 80,color, -1)
                #cv2.rectangle(img, (200*row,200*column), (200*row+200,200*column+200), color, -1)
    img=drawLines(img)
    cv2.imwrite(title+".png",img)

In [293]:
def drawLines(img):
    for i in [0,200,400,600,800]:
        cv2.line(img,(i,0),(i,800),(0,0,0),8)
        cv2.line(img,(0,i),(800,i),(0,0,0),8)
    return(img)

In [309]:
def play():
    state=""
    my_turn=True
    step=0
    removePngs("GUR")
    while 1:
        title=str(step)+"_"+state
        print(title)
        title="GUR@"+str(step)
        save(state,title=title)
        if checkStateStatus(state)=="WIN":
            if my_turn:
                print("Second player (RED) has won")
            else:
                print("First player (GREEN) has won")
            break
        if checkStateStatus(state)=="DRAW":
            print("Its a draw")
            break
        if not my_turn:
            state=getBestMove(state)
        else:
            #state=getRandomMove(state)
            state=getBestMove(state)
        step=step+1
        my_turn=not my_turn
    makeAnimation("GUR",filename="PLAY")
    #removePngs("GUR")

In [295]:
def getBestMove(s):
    if len(s)<16:
        s=s+"".join(["O" for i in range(16-len(s))])
    ltw=leadingToWin(s)
    if ltw!="NO":
        return(ltw)
    else:
        current_options=nextMove(s)
        np.random.shuffle(current_options)
        for state in current_options:
            if leadingToWin(state)=="NO":
                return(state)
        print("No Way to Win this")
        
        return(getRandomMove(s))

In [296]:
a=[1,2,3,4]
np.random.shuffle(a)


In [297]:
def leadingToWin(s):
    if len(s)<16:
        s=s+"".join(["O" for i in range(16-len(s))])
    
    availableMoves=nextMove(s)
    for state in availableMoves:
        if checkStateStatus(state)=="WIN":
            return(state)
    return("NO")

In [298]:
def getRandomMove(s):
    availableMoves=nextMove(s)
    return(np.random.choice(availableMoves))

In [299]:
def makeAnimation(title,filename="",wd="",extension="png",loop=True,delay=20):
     import imageio
     if len(filename)==0:
         filename=title
     print("Readying animation...", end=' ')
     import webbrowser
     if len(wd)==0:
            cwd = os.getcwd()
     else:
            #cwd=os.getcwd()+"/"+wd
             cwd=wd
     files=glob.glob(cwd+"/"+title+"@*."+extension)
     files.sort(key=os.path.getmtime)
     #files=[cwd+"/"+"b_"+str(i)+".jpeg" for i in range(37)]
     images=[]
     for file in files:
         images.append(imageio.imread(file))
     imageio.mimsave(filename+".gif", images,duration=0.75)


In [300]:
def removePngs(title):
    print("Deleting pngs and gifs...", end=' ')
    command='rm -rf '+title+"@*.png "
    os.system(command)
    command='rm -rf '+title+".gif "
    os.system(command)
    print("done.")


In [338]:
play()

Deleting pngs and gifs... done.
0_
1_FOOOOOOOOOOOOOOO
2_FOOOOOOSOOOOOOOO
3_FOFOOOOSOOOOOOOO
4_FOFOOOOSOOOSOOOO
5_FOFOOOOSOOOSOOFO
6_FOFOOSOSOOOSOOFO
7_FOFOFSOSOOOSOOFO
No Way to Win this
8_FOFOFSOSOOOSSOFO
9_FFFOFSOSOOOSSOFO
First player (GREEN) has won
Readying animation... 

In [317]:
def diagonize(s):
    save(s)
    arr=nextMove(s)
    a=[leadingToWin(state) for state in arr]
    print(a)